# Set Variables

In [ ]:
top_k = 20

label1_th = 0.5 # Th1

# -------- outp_version
output_version = 1092


# -------- method
# which_method = "third"

class_percentage = 0.5 # predictable classes (β)



# -------- method_way
method_way = "c" # a, b, c

if method_way == "a":
    bug_localization_accuracy = [1] # main data - th
    knn_accuracy_threshold =    [1]

elif method_way == "b":
    bug_localization_accuracy = [1] # main data - th
    knn_accuracy_threshold = [ 
        0.5, 0.55, 
        0.6, 0.65, 
        0.7, 0.75, 
        0.8, 0.85, 
        0.9, 0.95
    ]

else: # method_way == "c":
    bug_localization_accuracy = [0.4, 0.5, 0.6, 0.7, 0.8] # alpha (α)
    knn_accuracy_threshold = [ # theta (ϴ)
        0.5, 0.55, 
        0.6, 0.65, 
        0.7, 0.75, 
        0.8, 0.85, 
        0.9, 0.95
    ]


# -------- dataset
software_name = "Camel"
# software_name = "CloudStack"
# software_name = "Geode"
# software_name = "Hbase"

In [ ]:
print("bla: ", bug_localization_accuracy)
print("th:  ", knn_accuracy_threshold)

bla:  [0.4, 0.5, 0.6, 0.7, 0.8]
th:   [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


In [ ]:
if software_name == "Camel":
    dataset_file_name =    "Camel_Bugs - v.01"
    
elif software_name == "CloudStack":
    dataset_file_name =    "CloudStack_Bugs - v.01"
    
elif software_name == "Geode":
    dataset_file_name =    "Geode_Bug - v.01"
    
else:
    dataset_file_name =    "Hbase_Bugs - v.03"

# Google Colab

In [ ]:
!pip install enlighten
!pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# load data from google drive
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
!ls "/content/gdrive/My Drive/"

Mounted at /content/gdrive
'Colab Notebooks'


In [ ]:
# project folder path
project_folder = "gdrive/MyDrive/Colab Notebooks/paper/"

# data folder path
data_folder = "00- My Data/two-phase method/"

# dataset folder path
dataset_folder = software_name + "/"

# output folder path
output_folder = "01- Jupyter Notebook/80- two-phase method/00. Output/"

# Libs

In [ ]:
import random 
import json
import math
import os

import enlighten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import truediv
import seaborn as sns

# Config

In [ ]:
mypaths = {
    "data": {
        "dataset_bugs":          project_folder + data_folder + dataset_folder + dataset_file_name + ".csv",
        "dataset_bugs_bla_pred": project_folder + data_folder + dataset_folder   + "top-k/k={}/bla={}/" + "{} _ k={}, bla={} _ pred.csv",
    },
    "output": {
        "perfomance":            project_folder + output_folder + dataset_folder + dataset_file_name + "top-k/k={}/performance-v{}.json"
    }
}

preprocessing_params = {
    "data": {
        "dataset_bugs": {
            # time_stamp is the time that we change the class for a bug fixing.
            # id stands for identifier
            "columns_name":   ["bug_report_id", "class_id", "time_stamp", "label"],
            "columns_dtype" : {0: "int64", 1: "int64", 2: "int64", 3:"int32"}
        },
        "dataset_bugs_bla_pred": {
            # time_stamp is the time that we change the class for a bug fixing.
            # id stands for identifier
            "columns_name":   ["bug_report_id", "class_id", "time_stamp", "label"],
            "columns_dtype" : {0: "int64", 1: "int64", 2: "int64", 3:"int32"}
        },
    },
    
    # 0: no change | 1: increase bad smell | -1: decrease bad smell
    "class_labels": [0, 1, -1], 
    "num_classes": 3,
    "bc2_labels": [0, 1],
    "bc2_num_classes": 2,
    "top_k": None,
    "class_percentage": None,
    "label1_th": label1_th,
}

if (which_method == "first" or which_method == "third"):
    preprocessing_params["class_percentage"] = class_percentage

if (which_method == "second" or which_method == "third"):
    preprocessing_params["top_k"] = top_k
    
    

knn_model_params = {
    "train_size": 0.8,
    "n_neighbors": 5
}

progress_colours = ["webmaroon", "salmon", "orangered", "deeppink3", "crimson", 
                    "black", "blue", "gray", "cyan4", "darkgreen"]

In [ ]:
mypaths["data"]["dataset_bugs_bla_pred"] = mypaths["data"]["dataset_bugs_bla_pred"].format(
    top_k,
    "{}", # bla
    dataset_file_name,
    top_k,
    "{}" # bla
)

In [ ]:
mypaths["output"]["perfomance"] = mypaths["data"]["dataset_bugs_bla_pred"].format(
    top_k,
    output_version
)

# Read Files

## df_main

In [ ]:
df_main = pd.read_csv(
    mypaths["data"]["dataset_bugs"], 
    names=preprocessing_params["data"]["dataset_bugs"]["columns_name"], 
    dtype=preprocessing_params["data"]["dataset_bugs"]["columns_dtype"], 
    header=None
)

## df_data_pred_empty

In [ ]:
pred_col_name = []
pred_col_name.extend(preprocessing_params["data"]["dataset_bugs_bla_pred"]["columns_name"])
pred_col_type = {}
pred_col_type.update(preprocessing_params["data"]["dataset_bugs_bla_pred"]["columns_dtype"])
count = len(pred_col_type)
for th in knn_accuracy_threshold:
    temp1 = "predictions_" + str(th)
    pred_col_name.append(temp1)
    pred_col_type[count] = pd.Int32Dtype()
    count += 1

In [ ]:
pred_col_type

{0: 'int64',
 1: 'int64',
 2: 'int64',
 3: 'int32',
 4: Int32Dtype(),
 5: Int32Dtype(),
 6: Int32Dtype(),
 7: Int32Dtype(),
 8: Int32Dtype(),
 9: Int32Dtype(),
 10: Int32Dtype(),
 11: Int32Dtype(),
 12: Int32Dtype(),
 13: Int32Dtype()}

In [ ]:
print(len(pred_col_name))
pred_col_name

14


['bug_report_id',
 'class_id',
 'time_stamp',
 'label',
 'predictions_0.5',
 'predictions_0.55',
 'predictions_0.6',
 'predictions_0.65',
 'predictions_0.7',
 'predictions_0.75',
 'predictions_0.8',
 'predictions_0.85',
 'predictions_0.9',
 'predictions_0.95']

## df_data_pred

In [ ]:
df_data_pred = {}

for bla in bug_localization_accuracy:
    print("*" * 10, bla)
    full_path = mypaths["data"]["dataset_bugs_bla_pred"].format(
        bla,
        bla
    )
    print(full_path)
    print(os.path.isfile(full_path))
    df_data_pred[bla] = pd.read_csv(
        full_path, 
        dtype=pred_col_type
    )

********** 0.4
gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/Camel/top-k/k=20/bla=0.4/Camel_Bugs - v.01 _ k=20, bla=0.4 _ pred.csv
True
********** 0.5
gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/Camel/top-k/k=20/bla=0.5/Camel_Bugs - v.01 _ k=20, bla=0.5 _ pred.csv
True
********** 0.6
gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/Camel/top-k/k=20/bla=0.6/Camel_Bugs - v.01 _ k=20, bla=0.6 _ pred.csv
True
********** 0.7
gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/Camel/top-k/k=20/bla=0.7/Camel_Bugs - v.01 _ k=20, bla=0.7 _ pred.csv
True
********** 0.8
gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/Camel/top-k/k=20/bla=0.8/Camel_Bugs - v.01 _ k=20, bla=0.8 _ pred.csv
True


In [ ]:
df_data_pred

{0.4:        bug_report_id  class_id  time_stamp  label  predictions_0.5  \
 0                  1     12610  1582392860      1             <NA>   
 1                  1     35580  1582394536      0             <NA>   
 2                  1      2010  1582483410      0                1   
 3                  1      1782  1582483410      1             <NA>   
 4                  1      5899  1582483410      1             <NA>   
 ...              ...       ...         ...    ...              ...   
 45885           9656     37667  1183138957     -1             <NA>   
 45886           9656     37666  1183138957     -1             <NA>   
 45887           9658     37625  1182911537      1             <NA>   
 45888           9659     37625  1182853205      0             <NA>   
 45889           9660     37625  1182831028      1             <NA>   
 
        predictions_0.55  predictions_0.6  predictions_0.65  predictions_0.7  \
 0                  <NA>             <NA>              <NA>  

In [ ]:
dfg_brIDs = df_main.groupby("bug_report_id") # brIDs: bug report ids
brIDs_num = len(dfg_brIDs) # number of bug reports
brIDs_num

9024

# DataAfterTimeSerPrediction

In [ ]:
class DataAfterTimeSerPrediction:
    def __init__(self, df_data_pred, class_percentage, bug_localization_accuracy, threshold):
        self.class_percentage = class_percentage
        self.df_data_pred_ = df_data_pred
        self.df_data_pred_filtered = {}
        
        self.bla = bug_localization_accuracy
        self.threshold = threshold
    
    
    # ************************** progress lines ************************** #
    
    def progress_lines(self, colours):
        desc = self.set_strings_to_equal_len_()
        total = len(self.threshold)
        manager = enlighten.get_manager()
        progresses = {}
        for i, bla in enumerate(self.bla):
            prog = manager.counter(total=total, desc=desc[i], unit="sample", color=colours[i])
            prog.refresh()
            progresses[bla] = prog
        self.progresses = progresses
    
    
    def set_strings_to_equal_len_(self):
        words = []
        for bla in self.bla:
            desc = "filter-bla-" + str(bla)
            words.append(desc)
        
        max_len = 0
        longest_string_length = len(max(words, key=len))
        w = []
        for i, word in enumerate(words):
            temp = longest_string_length - len(word)
            w.append(word + " " * temp)
        return w
    
    
    # ************************** filter_data ************************** #
    
    def filter_data(self, which_method):
        for bla in self.bla:
            #print("*" * 10, bla)
            self.df_data_pred_filtered[bla] = []
            if (which_method == "first" or which_method == "third"):
                print("if ", which_method)
                for j, th in enumerate(self.threshold):
                    #print(th)
                    mydf = self.df_notna_base_prediction_th_column_(bla, th)
                    tempdf = mydf.groupby("bug_report_id")
                    tempdf.apply(lambda g: self.sub_1(g, mydf))
                    # ----
                    tempColumnName = self.prediction_column_name_(th)
                    mydf = mydf[mydf[tempColumnName].notna()]
                    self.df_data_pred_filtered[bla].append(mydf)
                    self.progresses[bla].update()
            else:
                print("else ", which_method)
                for j, th in enumerate(self.threshold):
                    #print(th)
                    mydf = self.df_notna_base_prediction_th_column_(bla, th)
                    # ----
                    tempColumnName = self.prediction_column_name_(th)
                    mydf = mydf[mydf[tempColumnName].notna()]
                    self.df_data_pred_filtered[bla].append(mydf)
                    self.progresses[bla].update()
    
    
    def df_notna_base_prediction_th_column_(self, bla, th):
        tempColumnName = self.prediction_column_name_(th)
        final_table_columns = ["bug_report_id", "label"] # ["bug_report_id", "label", "predictions_?"]
        final_table_columns.append(tempColumnName)
        
        tempDF = self.df_data_pred_[bla].copy()
        tempDF = tempDF[tempDF.columns.intersection(final_table_columns)]
        return tempDF
    
    
    def sub_1(self, g, mydf):
        numNan = g.iloc[:, -1].isna().sum()
        numPredictable = len(g) - numNan
        # ----
        temp3 = numPredictable / len(g)
        if temp3 < self.class_percentage:
            mydf.drop(g.index, inplace=True)
    
    
    def prediction_column_name_(self, th):
        return "predictions_" + str(th)
    
    
    def free_memory(self):
        self.df_data_pred_ = None

## obj

In [ ]:
datp_data = DataAfterTimeSerPrediction(
    df_data_pred, 
    preprocessing_params["class_percentage"], 
    bug_localization_accuracy, 
    knn_accuracy_threshold
)

datp_data.progress_lines(progress_colours)

datp_data.filter_data(
    which_method
)

datp_data.free_memory()

if  third


KeyboardInterrupt: ignored

In [ ]:
my_temp_100 = None
if knn_accuracy_threshold[0] == 1:
    my_temp_100 = datp_data.df_data_pred_filtered[1][0] # main data
else:
    print("else")
    if bug_localization_accuracy[0] == 1:
        print("bla = 1")
        my_temp_100 = datp_data.df_data_pred_filtered[1][0]
    else:
        print("bla != 1")
        my_temp_100 = datp_data.df_data_pred_filtered[0.4][0]

my_temp_100

In [ ]:
datp_data.df_data_pred_filtered

# BugClassSpecifying

In [ ]:
class BugClassSpecifying():
    
    def __init__(self, df, l1_th):
        # df: DataFrame("bug_report_id", "label|prediction_th")
        
        self.gdf = df.groupby("bug_report_id")
        self.l1_th = l1_th
        
        # df_bug_pred
        columns_name = ["bug_report_id", "bug_class_2"]
        columns_dtype = ["int64", "int32"]
        temp = {}
        for cn, cd in zip(columns_name, columns_dtype):
                temp[cn] = pd.Series(dtype=cd)
        self.df_bug_pred = pd.DataFrame(temp)
        self.df_bug_pred["bug_report_id"] = self.gdf.groups.keys()
        self.df_bug_pred["bug_class_2"] = self.df_bug_pred["bug_class_2"].astype("Int32")
    
    
    # =============================================
    #                   bug class
    # =============================================
    
    def bug_class(self):
        self.gdf.apply(lambda g: self.sub_bug_class_(g))
    
    
    def sub_bug_class_(self, g):
        # --------- bc_2: {1, 0}
        bc = None
        label_column = g.iloc[:, -1].to_list()
        label_column_num_ones = label_column.count(1)
        # ----
        temp3 = label_column_num_ones / len(label_column)
        bc = 1 if temp3 >= self.l1_th else 0
        self.df_bug_pred.loc[(self.df_bug_pred["bug_report_id"] == g.name), "bug_class_2"] = bc
    
    
    def free_memory(self):
        self.gdf = None

## obj

In [ ]:
df_main_bug_preds_obj = BugClassSpecifying(df_main.copy(), preprocessing_params["label1_th"])
df_main_bug_preds_obj.bug_class()
df_main_bug_preds_obj.free_memory()
df_main_bug_preds = df_main_bug_preds_obj.df_bug_pred
df_main_bug_preds

# BugPredictionForData

In [ ]:
class BugClassSpecifyingForData():
    def __init__(self, threshold, bug_localization_accuracy):
        self.df_data_bug_preds = {} # {"bla": [data_bug_th1, data_bug_th2, ...], ...}
        
        self.threshold = threshold
        self.bla = bug_localization_accuracy
    
    
    # ************************** progress lines ************************** #
    
    def progress_lines(self, colours):
        desc = self.set_strings_to_equal_len_()
        total = len(self.threshold)
        manager = enlighten.get_manager()
        progresses = []
        for i, bla in enumerate(self.bla):
            prog = manager.counter(total=total, desc=desc[i], unit="sample", color=colours[i])
            prog.refresh()
            progresses.append(prog)
        self.progresses = progresses
    
    
    def set_strings_to_equal_len_(self):
        words = []
        for bla in self.bla:
            desc = "bug-prediction-" + str(bla)
            words.append(desc)
        
        max_len = 0
        longest_string_length = len(max(words, key=len))
        w = []
        for i, word in enumerate(words):
            temp = longest_string_length - len(word)
            w.append(word + " " * temp)
        return w
    
    
    # ************************** make_bug_prediction ************************** #
    
    def make_bug_prediction(self, df_data_pred_filtered, l1_th):
        
        # df_data_bug_preds
        for i, bla in enumerate(self.bla):
            self.df_data_bug_preds[bla] = []
            for j, th in enumerate(self.threshold):
                tempBP = BugClassSpecifying(df_data_pred_filtered[bla][j], l1_th)
                tempBP.bug_class()
                self.df_data_bug_preds[bla].append(tempBP.df_bug_pred)
                self.progresses[i].update()

## obj

In [ ]:
bpfd_data = BugClassSpecifyingForData(
    knn_accuracy_threshold, 
    bug_localization_accuracy)

bpfd_data.progress_lines(progress_colours)
bpfd_data.make_bug_prediction(datp_data.df_data_pred_filtered, preprocessing_params["label1_th"])

In [ ]:
bpfd_data.df_data_bug_preds

In [ ]:
for i in bpfd_data.df_data_bug_preds:
    for j in bpfd_data.df_data_bug_preds[i]:
        print(len(j))

In [ ]:
len(bpfd_data.df_data_bug_preds)

# ClassDistribution

In [ ]:
class ClassDistribution:
    
    def __init__(self, df_bug_pred):
        self.df_bug_pred = df_bug_pred
        self.my_bug_classes = ["bug_class_2"]
        self.bug_label_num = {}
        self.bug_label_per = {}
        self.bug_label_ratio = {}
        
        for i, bug_class in enumerate(self.my_bug_classes):
            if i == 0: # bug_label_2
                self.bug_label_num[bug_class] =   {"0": 0, "1":  0}
                self.bug_label_per[bug_class] =   {"0": 0, "1":  0}
                self.bug_label_ratio[bug_class] = {}
        
        # plot values
        self.colours = ["red", "blue", "green", "cyan"]
    
    
    # --------------------------- calc_bug_label_num
    
    def calc_bug_label_num_per(self):
        self.df_bug_pred.groupby("bug_class_2").apply(lambda g: self.sub_bln_(g, "bug_class_2"))
    
    
    def sub_bln_(self, g, gby):
        self.bug_label_num[gby][str(g.name)] = len(g)
        self.bug_label_per[gby][str(g.name)] = (len(g) / len(self.df_bug_pred)) * 100
    
    
    # --------------------------- calc_ratios
    
    def calc_bug_label_ratios(self):
        
        for bug_class in self.my_bug_classes:
            temp = self.bug_label_num[bug_class]
            mydata_sorted = [(key, value) for key, value in sorted(temp.items(), 
                                                                   key=lambda item: item[1], reverse=True)]
        
            num_classes = len(temp)
            for i in range(num_classes - 1):
                for j in range(i + 1, num_classes):
                    tempKey1 = mydata_sorted[i][0]
                    tempKey2 = mydata_sorted[j][0]
                    mykey = tempKey1 + "/" + tempKey2

                    tempValue1 = mydata_sorted[i][1]
                    tempValue2 = mydata_sorted[j][1]
                    if tempValue2 != 0:
                        self.bug_label_ratio[bug_class][mykey] = tempValue1 / tempValue2
    
    
    # --------------------------- bar plot
    
    def plot_percent_num(self, bla, th, num_bugs):
        # figsize=(a, b) : a_width, b_height
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
        width = 0.2
        my_x = [-0.3, -0.1, 0.1, 0.3]
        
        temp1 = str(num_bugs)
        temp2 = len(temp1) - 1
        temp3 = "1" + "0" * temp2
        temp4 = temp1[0] + "0" * temp2
        temp5 = int(temp3) + int(temp4)
        
        ml = list( range(0, temp5, int(temp3)) )
        ml.append(num_bugs)
        
        for i, key in enumerate(self.my_bug_classes):
            ax1 = axes[0]
            ax2 = axes[1]
            for j, cl in enumerate(self.bug_label_per[key]):
                ax1.bar(my_x[j], self.bug_label_num[key][cl], width=width, label=cl, color=self.colours[j])
                ax2.bar(my_x[j], self.bug_label_per[key][cl], width=width, label=cl, color=self.colours[j])
            
            tempXlabel = "{}".format(key)
            ax1.set_xlabel(tempXlabel)
            ax2.set_xlabel(tempXlabel)
            
            ax1.set_ylabel("Number of Bug Reports")
            ax2.set_ylabel("Percentages (%)")
            
            tempXlim = [-0.5, 0.5]
            ax1.set_xlim(tempXlim)
            ax2.set_xlim(tempXlim)
            
            
            ax1.set_ylim([0, temp5])
            ax2.set_ylim([0, 110])
            
            
            ax1.get_xaxis().set_ticks([])
            ax2.get_xaxis().set_ticks([])
            
            
            ax1.get_yaxis().set_ticks(ml)
            ax2.get_yaxis().set_ticks(list(range(0, 101, 10)))
            
            ax1.legend()
            ax2.legend()
            
            ax1.grid(axis="y")
            ax2.grid(axis="y")
        
        fig.suptitle("bla: {}, th: {}".format(bla, th), fontsize=16)
        fig.tight_layout(pad=3.0)
    
    
    def free_memory(self):
        self.df_bug_pred = None

# ClassDistributionForData

In [ ]:
class ClassDistributionForData():
    def __init__(self, threshold, bug_localization_accuracy):
        self.bla = bug_localization_accuracy
        self.threshold = threshold
        
        self.df_data_bug_class_distribution = {}
        for bla in self.bla:
            self.df_data_bug_class_distribution[bla] = {}
            for th in self.threshold:
                self.df_data_bug_class_distribution[bla][th] = {
                    "bug_class_num": None,
                    "bug_class_per": None,
                    "bug_class_ratio": None
                }
    
    
    def plotm(self, df_main_bug_preds, df_data_bug_preds):
        # df_data_bug_plot
        for i, bla in enumerate(self.bla):
            for j, th in enumerate(self.threshold):
                #print(j)
                temp1 = df_data_bug_preds[bla][j]
                temp1_brid = temp1["bug_report_id"].to_list()
                temp2 = df_main_bug_preds[df_main_bug_preds["bug_report_id"].isin(temp1_brid)]
                
                tempBP = ClassDistribution(temp2.copy())
                tempBP.calc_bug_label_num_per()
                tempBP.calc_bug_label_ratios()
                tempBP.plot_percent_num(bla, th, len(temp2))
                tempBP.free_memory()
                
                self.df_data_bug_class_distribution[bla][th]["bug_class_num"] = tempBP.bug_label_num
                self.df_data_bug_class_distribution[bla][th]["bug_class_per"] = tempBP.bug_label_per
                self.df_data_bug_class_distribution[bla][th]["bug_class_ratio"] = tempBP.bug_label_ratio

## obj

In [ ]:
cddd = ClassDistributionForData(
    knn_accuracy_threshold, 
    bug_localization_accuracy)

In [ ]:
cddd.plotm(
    df_main_bug_preds, 
    bpfd_data.df_data_bug_preds)

In [ ]:
cddd.df_data_bug_class_distribution

# CalcAcc

In [ ]:
class MyConfusionMatrix():
    def __init__(self, num_classes):
        # rows: actual, columns: prediction
        self.confusion_matrix = np.zeros((num_classes, num_classes), dtype=np.int32)
    
    
    def update(self, y, yhat_indices):
        for actual, pred in zip(y, yhat_indices):
            self.confusion_matrix[actual, pred] += 1
    
    
    def calc_accuracy(self):
        diagon = self.confusion_matrix.diagonal()
        # accuracy
        total_samples = self.confusion_matrix.sum()
        total_corrects = diagon.sum()
        accuracy = 100 * (total_corrects / total_samples)
        
        # accuracy per class
        # sum(1): 1 referes to sum for each row
        samples_per_class = self.confusion_matrix.sum(1)
        accuracy_per_class = 100 * (np.divide(diagon, samples_per_class))
        
        return accuracy, accuracy_per_class.tolist()
    
    
    def get_cf(self):
        return self.confusion_matrix.tolist()

In [ ]:
class CalcAcc():
    
    def __init__(self, df_main_bug_preds, threshold, bug_localization_accuracy, num_bugc2_classes):
        self.confusion_matrix = {}
        self.mycf = {}
        
        self.df_main_bug_preds = df_main_bug_preds
        self.threshold = threshold
        self.bla = bug_localization_accuracy
        
        self.samples_num = {}
        self.num_true = {} # {"0.50": {"simple": 100, "bc_2": 200, "bc_3":200, "bc_4":200}, ...}
        self.accuracy = {} # {"0.50": {"simple": 92.0, "bc_2": 100, "bc_3":100, "bc_4":100}, ...}
        self.brIDs_num = {}
        
        for bla in bug_localization_accuracy:
            self.samples_num[bla] = {}
            self.num_true[bla] = {}
            self.accuracy[bla] = {}
            self.brIDs_num[bla] = {}
            #--
            self.confusion_matrix[bla] = {}
            self.mycf[bla] = {}
            
            for th in threshold:
                self.num_true[bla][th] = {
                    "simple": 0, 
                    "bc_2": 0,
                }
            
                # bc: bug class
                self.accuracy[bla][th] = {
                    "simple": None, 
                    "bc_2": None, 
                }
                
                self.mycf[bla][th] = {}
                self.confusion_matrix[bla][th] = MyConfusionMatrix(num_bugc2_classes)
    
    
    # ================== simple ==================
    
    def simple(self, df_data_pred_filtered):
        for i, bla in enumerate(self.bla):
            for j, th in enumerate(self.threshold):
                df_data_pred_filtered[bla][j].apply(lambda row: self.sub_simple_(row, bla, th), axis=1)
                self.samples_num[bla][th] = len(df_data_pred_filtered[bla][j])
                print(bla, th, self.samples_num[bla][th])
                self.accuracy[bla][th]["simple"] = (self.num_true[bla][th]["simple"] / self.samples_num[bla][th]) * 100
    
    
    def sub_simple_(self, row, bla, th):
        tem = row.tail(1).item()
        if row["label"] == tem:
            self.num_true[bla][th]["simple"] += 1
    
    
    # =============================================
    #                   bug class
    # =============================================
    
    def bug_class(self, df_data_bug_preds):
        for i, bla in enumerate(self.bla):
            for j, th in enumerate(self.threshold):
                self.brIDs_num[bla][th] = len(df_data_bug_preds[bla][j])
                df_data_bug_preds[bla][j].apply(lambda row: self.sub_bug_class_(row, bla, th), axis=1)
                
                
                my_accuracy, acc_per = self.confusion_matrix[bla][th].calc_accuracy()
                self.accuracy[bla][th]["bc_2"] = my_accuracy
                # ----
                
                self.mycf[bla][th]["confusion_matrix"] = self.confusion_matrix[bla][th].get_cf()
                self.mycf[bla][th]["acc_per_class"] = acc_per
    
    
    def sub_bug_class_(self, row, bla, th):
        temp = self.df_main_bug_preds[(self.df_main_bug_preds["bug_report_id"] == row["bug_report_id"])]
        
        if row["bug_class_2"] == temp["bug_class_2"].item():
            self.num_true[bla][th]["bc_2"] += 1
        
        # confusion_matrix.update(y, yhat_indices)
        self.confusion_matrix[bla][th].update([temp["bug_class_2"].item()], [row["bug_class_2"]])

## obj

In [ ]:
ca = CalcAcc(df_main_bug_preds, knn_accuracy_threshold, bug_localization_accuracy, 2)
ca.simple(datp_data.df_data_pred_filtered)
ca.bug_class(bpfd_data.df_data_bug_preds)

In [ ]:
ca.accuracy

In [ ]:
ca.mycf

In [ ]:
bpfd_data.df_data_bug_preds

# Save results

In [ ]:
def save_to_file_results(
    which_method,
    dataset_name, 
    preprocessing_params, 
    bug_localization_accuracy, 
    knn_model_params, 
    knn_accuracy_threshold, 

    samples_num, 
    brIDs_num, 

    samples_num_pred, 
    brIDs_num_pred, 

    df_data_bug_class_distribution,
    accuracies, 
    
    cf,
    
    result_path):
    
    tempStructure = {
        "which_method":                   which_method,
        "dataset":                        dataset_name, 
        "preprocessing_params":           preprocessing_params, 
        "bug_localization_accuracy":      bug_localization_accuracy, 
        "knn_model_params":               knn_model_params,
        "knn_accuracy_threshold":         knn_accuracy_threshold,
        
        "samples_num":                    samples_num,
        "brIDs_num":                      brIDs_num,
        
        "samples_num_pred":               samples_num_pred,
        "brIDs_num_pred":                 brIDs_num_pred,
        
        "df_data_bug_class_distribution": df_data_bug_class_distribution,
        "accuracies":                     accuracies, 
        
        "confusion_matrix":               cf,
        
    }

    with open(result_path, "w") as fout:
        json.dump(tempStructure, fout)

## call

In [ ]:
save_to_file_results(
    which_method,
    dataset_file_name, 
    preprocessing_params, 
    bug_localization_accuracy, 
    knn_model_params, 
    knn_accuracy_threshold, 

    len(df_data_pred[0.8]), 
    brIDs_num, 
    
    ca.samples_num, 
    ca.brIDs_num, 

    cddd.df_data_bug_class_distribution,
    ca.accuracy, 
    
    ca.mycf,
    
    mypaths["output_performance"]
)